In [1]:
import sys

import timeit
import tensorflow as tf
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

sys.path.append('/home/hoheon/packages/')

from utils.keras import get_model_memory_usage
from utils.numpy import to_onehot

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [2]:
# Given data
train, test = tf.keras.datasets.cifar10.load_data()
tf.config.list_physical_devices()
x_train, y_train = train[0], train[1]
x_test, y_test = test[0], test[1]

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

y_train = to_onehot(y_train, 10).reshape(-1, 10)
y_val = to_onehot(y_val, 10).reshape(-1, 10)
y_test = to_onehot(y_test, 10).reshape(-1, 10)

print(x_train.shape, y_train.shape)
print("N classes: {}".format(y_test.shape[-1]))

(35000, 32, 32, 3) (35000, 10)
N classes: 10


##### Model

In [ ]:
tf.debugging.set_log_device_placement(True)

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    sub_model = tf.keras.applications.VGG19(include_top=False, input_shape=(32, 32, 3), classes=10)
    sub_model.trainable = False

    flat = tf.keras.layers.Flatten()(sub_model.layers[-1].output)
    classify_ = tf.keras.layers.Dense(10, activation='softmax')(flat)
    model = tf.keras.Model(inputs=sub_model.inputs, outputs=classify_)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.05), loss='categorical_crossentropy', metrics=['accuracy'])
    
    lr_decay = tf.keras.callbacks.ReduceLROnPlateau()
    check_pt = tf.keras.callbacks.ModelCheckpoint('./cache/best_param.hdf5', monitor='val_loss', save_best_only=True)

    EPOCHS = 500
    BATCH_SIZE = 300
    CB = [lr_decay, check_pt]


    history  = model.fit(x=x_train, 
                          y=y_train,
                          epochs=EPOCHS,
                          validation_data=(x_val,  y_val),
                          batch_size=BATCH_SIZE,
                          callbacks=CB,
                          verbose=False)

In [ ]:
sub_model = tf.keras.applications.VGG19(include_top=False, input_shape=(32, 32, 3), classes=10)
sub_model.trainable = False

flat = tf.keras.layers.Flatten()(sub_model.layers[-1].output)
classify_ = tf.keras.layers.Dense(10, activation='softmax')(flat)
model = tf.keras.Model(inputs=sub_model.inputs, outputs=classify_)


model.load_weights('./cache/best_param.hdf5')

##### Model Performance

In [ ]:
from utils.numpy import to_onehot
from sklearn.metrics import multilabel_confusion_matrix

y_hat = model.predict(x_test)
y_label = to_onehot(np.argmax(y_hat, axis=1), 10)


cm = multilabel_confusion_matrix(y_test, y_label)

##### Runtime

In [ ]:
def layer_runtime(index, batch, model, repeat=150, return_size=False):
    '''
    Parameters
    ----------
    index: int:
        layer index
    model: keras.model
    repeate: int
    
    Return
    ------
    runtime: [sec, sec, sec...] 
    dsize: /kbyte
    '''
    
    
    front_model = tf.keras.Model(model.input, model.layers[index-1].output)
    front_output = front_model(batch)
    
    runtimes = []
    for _ in range(repeat):
        start_time = timeit.default_timer()
        model.layers[index](front_output)
        end_time = timeit.default_timer()
        
        runtime = end_time - start_time
        runtimes.append(runtime)
    
    if return_size == False:
        return runtimes
    else:
        return_val = model.layers[index](front_output)
        mem_size = return_val.numpy().nbytes / 1024
        return runtimes, mem_size


resource_consump = dict()

for i in range(0, len(model.layers)):
    runtimes, d_size = layer_runtime(i, x_train[0:1], model, repeat=50, return_size=True)
    
    l_name = model.layers[i].name
    resource_consump[l_name] = {'runtime':runtimes, 'dsize':d_size}

In [ ]:
df = []

for layer_name in list(resource_consump.keys()):
    _df = pd.DataFrame(resource_consump[layer_name]['runtime'])
    _df['layer_name'] = layer_name
    df.append(_df)
    
runtimes = pd.concat(df)
runtimes.columns = ['runtime', 'layer_name']
runtimes.head(2)

In [ ]:
df = []
for layer_name in list(resource_consump.keys()):
    _dsize = resource_consump[layer_name]['dsize']
    df.append([layer_name, _dsize])
    
d_size = pd.DataFrame(df)
d_size.columns = ['layer_name', 'dsize']
d_size.head(2)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(figsize=(10 ,10))
sns.boxplot(y='runtime', x='layer_name', data=runtimes, orient='v')


plt.ylim(-0.0001, 0.001)
plt.xticks(rotation=80)

ax2 = plt.twinx()
sns.lineplot(x='layer_name', y='dsize', data=d_size, ax=ax2, markers='*', color='red', linestyle='--')

In [3]:
class VGG19EX(tf.keras.Model):
    
    def __init__(self, exit_point):
        super(VGG19EX, self).__init__()
        
        # Instance variables with as global param in class
        self.EXITPT = exit_point
        
        # subclass
        self.base_model = tf.keras.applications.VGG19(include_top=False, input_shape=(32, 32, 3), classes=10)
        self.base_model.trainable = False
        self.graph = self.build_graph_by_sub()
        
        n_layers= len(self.base_model.layers)
        if self.EXITPT not in range(0, n_layers):
            raise ValueError('exit_point must be in range(0, 22)')
            
    
    def summary(self):
        stringlist = []
        self.graph.summary(print_fn=lambda x: stringlist.append(x))
        stringlist_summary = "\n".join(stringlist)
        print(stringlist_summary)
        
    def build_graph_by_sub(self):
        sub_graph = tf.keras.models.Model(self.base_model.input, 
                                          self.base_model.layers[self.EXITPT].output)
        
        _flatten = tf.keras.layers.Flatten()(sub_graph.output)
        _fc = tf.keras.layers.Dense(10, activation='softmax')(_flatten)
        trimed_graph = tf.keras.Model(inputs=sub_graph.input, outputs=_fc)
        return trimed_graph
    
    def call(self, x):  
        x = self.graph(x)
        return x
    
    def __repr__(self):
        _ =  self.build_graph_by_sub()
        print (_.summary())
    
    
vgg = VGG19EX(exit_point=1)
vgg.compile(optimizer=tf.keras.optimizers.Adam(0.05), loss='categorical_crossentropy', metrics=['accuracy'])
vgg.fit(x_train, y_train, epochs=10)
vgg.summary()

Epoch 1/10
1094/1094 [==============================] - 2s 2ms/step - loss: 32803.8281 - accuracy: 0.3815
Epoch 2/10
1094/1094 [==============================] - 2s 2ms/step - loss: 29497.9297 - accuracy: 0.4665
Epoch 3/10
1094/1094 [==============================] - 2s 2ms/step - loss: 30720.2090 - accuracy: 0.4920
Epoch 4/10
1094/1094 [==============================] - 2s 2ms/step - loss: 28989.8340 - accuracy: 0.5187
Epoch 5/10
1094/1094 [==============================] - 2s 2ms/step - loss: 28091.7773 - accuracy: 0.5396
Epoch 6/10
1094/1094 [==============================] - 2s 2ms/step - loss: 27951.6934 - accuracy: 0.5511
Epoch 7/10
1094/1094 [==============================] - 2s 2ms/step - loss: 26218.1602 - accuracy: 0.5693
Epoch 8/10
1094/1094 [==============================] - 2s 2ms/step - loss: 25467.9277 - accuracy: 0.5832
Epoch 9/10
1094/1094 [==============================] - 2s 2ms/step - loss: 24908.2109 - accuracy: 0.5868
Epoch 10/10
1094/1094 [=======================

In [4]:
tf.debugging.set_log_device_placement(True)

strategy = tf.distribute.MirroredStrategy()

history_master = {}

for i in range (1, 22):
    with strategy.scope():

        vgg = VGG19EX(exit_point=i)
        vgg.compile(optimizer=tf.keras.optimizers.Adam(0.05), loss='categorical_crossentropy', metrics=['accuracy'])

        lr_decay = tf.keras.callbacks.ReduceLROnPlateau()
        check_pt = tf.keras.callbacks.ModelCheckpoint('./weights/best_param_exit{}.hdf5'.format(i),
                                                      monitor='val_loss', 
                                                      save_best_only=True)

        EPOCHS = 500
        BATCH_SIZE = 300
        CB = [lr_decay, check_pt]


        history  = vgg.fit(x=x_train, 
                          y=y_train,
                          epochs=EPOCHS,
                          validation_data=(x_val,  y_val),
                          batch_size=BATCH_SIZE,
                          callbacks=CB,
                          verbose=False)
        
        history_master['exit{}'.format(i)] = history
    
    print("The model of exit point {} was learn".format(i))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhos

In [ ]:
get_model_memory_usage(1, model)
